In [6]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install statsmodels
# !pip install sklearn

Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'


You should consider upgrading via the 'c:\users\egorm\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle

In [8]:
from statsmodels.api import OLS
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

In [18]:
with open('dataset.pkl', 'rb') as f:
    data = pickle.load(f)

In [24]:
new_data = data['MutantApeYachtClub'].copy()
new_data = new_data.dropna()
new_data.head()

,dates,prices_avg,prices_floor,prices_median,volume,marketcap,lowwatermark,amount_transactions,amount_owners,amount_sellers,amount_buyers,btc adj close,btc volume,eth adj close,eth volume,sol adj close,sol volume,change in btc,change in eth,change in sol
1,2021-08-30,42150.432895,0.000000,32023.800781,4.611257e+07,3.916317e+06,20522.01,1094,7503.0,2.0,46.0,47054.984375,3.184701e+10,3224.374268,1.930692e+10,110.544807,5.274104e+09,-1774.847656,-2.628418,16.075134
2,2021-08-31,34728.502238,0.000000,28324.003906,1.972579e+07,7.019255e+06,20522.01,568,7693.0,3.0,24.0,47166.687500,3.473036e+10,3433.732666,2.728050e+10,108.483772,7.590894e+09,111.703125,209.358398,-2.061035
3,2021-09-01,34049.167712,1848.198730,24513.210938,1.637765e+07,2.655862e+07,20522.01,481,7793.0,6.0,42.0,48847.027344,3.913940e+10,3834.828125,3.007089e+10,111.033005,3.974444e+09,1680.339844,401.095459,2.549232
4,2021-09-02,26299.774417,12414.097656,22857.037109,1.041471e+07,1.794458e+08,20522.01,396,7933.0,9.0,21.0,49327.722656,3.950807e+10,3790.989990,2.438740e+10,128.102386,5.074400e+09,480.695312,-43.838135,17.069382
5,2021-09-03,27429.236859,16469.503906,23066.734375,7.159031e+06,2.389890e+08,20522.01,261,7992.0,3.0,32.0,50025.375000,4.320618e+10,3940.614746,2.620777e+10,146.583054,7.526374e+09,697.652344,149.624756,18.480667


In [26]:
y = new_data['prices_avg'].copy()
x = new_data.drop(['prices_floor', 'prices_avg', 'prices_median'], axis=1).copy() 

In [29]:
y

1      42150.432895
2      34728.502238
3      34049.167712
4      26299.774417
5      27429.236859
           ...     
266    32168.854391
267    31780.733774
268    31283.230239
269    36870.326782
270    36696.986979
Name: prices_avg, Length: 270, dtype: float64